In [281]:
#### Se importan todas las librerías necesarias ####
import plotly as px                         # Cosas de plotly generales. Como los colores
from plotly.offline import iplot, init_notebook_mode
from plotly.subplots import make_subplots
import plotly.graph_objs as go              # Para generar las figuras y los charts
import plotly.io as pio                     # Interfaz para guardar en SVG
import os                                   # Para crear carpetas
import pandas as pd                         # Para leer el CSV con los datos
import math

In [282]:
# You can also plot your graphs offline inside a Jupyter Notebook Environment. First you need to initiate the Plotly Notebook mode as below:
init_notebook_mode(connected=True) # Hola

In [283]:
#### Se cargan los datos en CSV con la librería Pandas ####
data = pd.read_csv(
    "test_data.csv",
    sep=';',        # Por defecto es ,
    decimal=",",     # Por defecto es .
)
dataTest = pd.read_csv(
    "dataMeanTest.csv",
    sep=';',        # Por defecto es ,
    decimal=",",     # Por defecto es .
)

In [284]:
# function below sets the color based on amount
def SetMarkerForm(x):
    Symbol1=0 # Imbalanced
    Symbol2=5 # Random
    Symbol3=2 # SMOTE

    if(x=="Imbalanced"):
        return Symbol1
    elif(x=="random undersampling"):
        return Symbol2
    else:
        return Symbol3

# function below sets the color based on amount
def SetMarkerColor(x):
    color1=px.colors.qualitative.T10[0] # Complete - blue
    color2=px.colors.qualitative.T10[2] # No cleaning - red
    color3=px.colors.qualitative.T10[0] # No filtering - orange
    color4=px.colors.qualitative.T10[0] # No segment - celeste
    color5=px.colors.qualitative.T10[0] # Reduced inputs - green

    if(x=="Complete"):
        return color1
    elif(x=="No cleaning"):
        return color2
    elif(x=="No filtering"):
        return color3
    elif(x=="No segment"):
        return color4
    else:
        return color5

# function below sets the color based on amount
def SetBorderMarkerColor(x):
    borderColorSVM  = px.colors.qualitative.T10[7]    # Pastel pink
    borderColorTree = px.colors.qualitative.Dark24[5] # Black
    if(x=="SVM"):
        return borderColorSVM
    elif(x=="Tree"):
        return borderColorTree
    else:
        return borderColorTree

FmeasureColor = px.colors.qualitative.T10[2]


fig = go.Figure()

for i in range(len(data)):
    idxMean = max( 0, math.floor((i-1)/10) )
    markerSel   = SetMarkerForm(data.balanceType[i])
    colorSel    = SetMarkerColor(data.Setting[i])
    borderColor = SetBorderMarkerColor(data.classifierType[i])

    fig.add_trace( # Accuracy K-Fold Mean
        go.Bar(
            x=[ [data.Name.values[i]], [data.foldExecuted.values[i]] ],
            y=[data.MeanAccuracy.values[i]],
            name="MeanAccuracy",
            width=10,
            offset=-0.5,
            marker_color=colorSel,
            marker_line_color='rgb(8,48,107)',
            marker_line_width=1.5,
            opacity=0.6,
            showlegend=True if i==1 else False,
        )
    )
    fig.add_trace( # Accuracy each k-fold
        go.Bar(
            x=[ [data.Name.values[i]], [data.foldExecuted.values[i]] ],
            y=[data.accuracy.values[i]],
            name="Accuracy",
            width=0.7,
            marker_color=colorSel,
            showlegend=True if i==1 else False,
        )
    )
    fig.add_trace( # F-measure K-Fold Mean
        go.Bar(
            x=[ [data.Name.values[i]], [data.foldExecuted.values[i]] ],
            y=[data.MeanFMeasure.values[i]],
            name="MeanFmeasure",
            width=10,
            offset=-0.5,
            marker_color=FmeasureColor,
            marker_line_color='rgb(8,48,107)',
            marker_line_width=1.5,
            opacity=0.6,
            showlegend=True if i==1 else False,
        )
    )
    
    fig.add_trace( # F-measure each k-fold
        go.Bar(
            x=[ [data.Name.values[i]], [data.foldExecuted.values[i]] ],
            y=[data.Fmeasure.values[i]*100],
            name="Fmeasure",
            width=0.7,
            marker_color=FmeasureColor,
            showlegend=True if i==1 else False,
        )
    )

    if (i % 10) == 5:
        fig.add_trace( # LINEA Accuracy Test
            go.Scatter(
                x=[ [data.Name.values[i]], [data.foldExecuted.values[i]] ],
                y=[dataTest.TestAccuracy.values[idxMean]],
                mode='markers',
                marker={
                    'symbol': markerSel,
                    'color' : colorSel,
                    'size'  : 10,
                    'opacity': 0.75,
                    'line'  : {
                        'width': 1.5,
                        'color': borderColor,
                    },
                },
            )
        )
        fig.add_trace( # LINEA F-measure Test
            go.Scatter(
                x=[ [data.Name.values[i]], [data.foldExecuted.values[i]] ],
                y=[dataTest.TestFmeasure.values[idxMean]],
                mode='markers',
                marker={
                    'symbol': markerSel,
                    'color' : FmeasureColor,
                    'size'  : 10,
                    'opacity': 0.75,
                    'line'  : {
                        'width': 1.5,
                        'color': borderColor,
                    },
                },
            )
        )

fig.update_layout(
    barmode='overlay',
    #showlegend=False,
    width=1000,
    height=400,
    xaxis= {
        'linecolor': 'black',
        #'linewidth': 1,
        'mirror': True,
        'title_text': "K-fold test - Experiment",
        "range":      [ -1, 60 ],
        "tickfont": {
            "size":   10
        }
    },
    yaxis= {
        'linecolor': 'black',
        #'linewidth': 1,
        'mirror':     True,
        'title_text': "Fmeasure - Accuracy",
        'tick0':      0,
        'dtick':      5,
        "range":      [ 0, 85 ],
    },
    margin={
        'l':   10,
        'r':   10,
        'b':   10,
        't':   10,
        'pad': 2
    },
    #bargap=0.015, # gap between bars of adjacent location coordinates.
    #bargroupgap=0.01 # gap between bars of the same location coordinate.
)

fig.show() # Para que salga el resultado
pio.write_image(fig, "figura.svg")